In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey

from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
import functions_pebble_accretion as peb
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as ub
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

import matplotlib.lines as mlines 

color = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))


In [ ]:
from matplotlib import pyplot as plt, ticker as mticker

# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                #'M_dot_star': 1e-8*const.M_sun.cgs/(1*u.yr).to(u.s),
                'iceline_radius': None,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'alpha': 1e-2,
                'v_frag': (1 * u.m/u.s).to(u.au/u.Myr).value,
                'iceline_alpha_change': False,
                'iceline_flux_change': False
                }

r = np.geomspace(3e-3, 1e2, num = 1000)
color = [ 'slateblue', 'purple', 'gold', 'orange', 'mediumaquamarine']
t = 0.1 #will be ignored because I set the Mdot_*
t_in = 0.1
t_fin = 10

model_label = [ "Ida et al. 2016", "Liu et al. 2019", "irradiated"]
ls = '-'
ls_gas = '-.'

fig = plt.figure( figsize = (9, 11))
gs = fig.add_gridspec(2, 1, hspace=0, wspace=0)
(axs0, axs1) = gs.subplots(sharex='col')


epsilon_el = [1e-2, 1]
epsilon_heat = [0.5, 1]
lam_mod=['surface-heated', 'midplane-heated']
for j, (e_el, e_h) in enumerate(zip(epsilon_el, epsilon_heat)):
    params = code_gas.Params(**params_dict, H_r_model = "Lambrechts_mixed", epsilon_el=e_el ,epsilon_heat=e_h)
    T = 1200
    si_subl_line = iceline(t, T, params)
    si_subl_point = np.argmin(r < si_subl_line)

    axs1.scatter(si_subl_line, sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)[si_subl_point], color = 'red', label = "Si sublimation front" if j==0 else None, zorder = 20)

    axs0.loglog(r, H_R(r, t, params), color = color[j], 
                  label = "this study (" + lam_mod[j]+"), \n$\epsilon_{el}$ = "+str(e_el)+", $\epsilon_{heat}$ = "+str(e_h), linestyle = ls) 
    print(params.epsilon_el, params.epsilon_heat)

    axs1.loglog(r[si_subl_point:], sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)[si_subl_point:], color = color[j], linestyle = ls)
    axs1.loglog(r, sigma_gas_steady_state(r, t, params), color = color[j], linestyle = ls_gas)
    axs1.loglog(r, MMSN(r), color = 'black', linestyle = ':', alpha = 0.5, zorder = 0, label = "MMSN, Hayashi et al. 1981" if j == 0 else None)

for j, model in enumerate(["Ida_mixed", "Liu_mixed","irradiated"]):
    params = code_gas.Params(**params_dict, H_r_model = model)
    print(params.H_r_model)
    T = 1200
    si_subl_line = iceline(t, T, params)
    si_subl_point = np.argmin(r < si_subl_line)
    axs1.scatter(si_subl_line, sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)[si_subl_point], color = 'red',  zorder = 20)
    axs0.loglog(r, H_R(r, t, params), color = color[j+2], label = model_label[j], linestyle = ls)
    axs1.loglog(r[si_subl_point:], sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params)[si_subl_point:], 
                  color = color[j+2], label = "$\Sigma_{\mathrm{peb}}$" if j == 2 else None, linestyle = ls)
    axs1.loglog(r, sigma_gas_steady_state(r, t, params), color = color[j+2], 
                  label = "$\Sigma_{\mathrm{gas}}$" if j == 2 else None, linestyle = ls_gas)
 


for axs in [axs0, axs1]:
    axs1.set_xlabel('r [AU]', size = 25)
    axs.tick_params(axis = "both", length = 10, which = "minor", direction = 'in')
    axs.tick_params(axis = "both", length = 15, which = "major", direction = 'in', labelsize = 15)
    axs.legend(fontsize = 14)
    #axs[i].axvline(1, color = 'lightgrey', linestyle = ':', alpha = 0.5, zorder = 0)
    axs.axvline(r_magnetic_cavity(t_in, params), linestyle = '-.', color = 'grey', alpha = 0.5)
    axs.set_xlim(3e-3, 1e2)
    axs.yaxis.set_major_locator(mticker.LogLocator(numticks=999))
    axs.yaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs="auto"))

axs0.text(r_magnetic_cavity(t_in, params)+4e-3, 1.5e-3, '$r_{\mathrm{mag. cav.}}$', fontsize=15, color='grey', rotation=90, rotation_mode='anchor', ha='left', va='bottom', zorder = 0)
axs1.text(r_magnetic_cavity(t_in, params)+4e-3, 1e-1, '$r_{\mathrm{mag. cav.}}$', fontsize=15, color='grey', rotation=90, rotation_mode='anchor', ha='left', va='bottom', zorder = 0)

axs0.set_ylabel('H/r', size = 25)
#axs1.set_ylabel('$\Sigma \: \mathrm{[g/cm^2]}$', size = 25)
axs1.set_ylabel('$\Sigma \: \mathrm{[M_{\oplus}/AU^2]}$', size = 25)

#axs[2].set_ylim(1e-1, 1e7)
axs0.set_ylim(1e-3, 3e-1)

axs0.legend(loc= (0.50, 0.06), fontsize=14)
axs0.set_title(" $t_0$ = "+str(t)+" Myr", fontsize = 19)
#axs[1].set_title(" Surface densities at $t_0$ = "+str(t.to(u.Myr)), fontsize = 19)
# axs[0].set_title(" H/R profiles for $\dot{M}_{*} = 10^{-8} M_{\odot}/yr$, "+r"$\alpha$="+str(params.alpha), fontsize = 15)
# axs[1].set_title(" Midplane temperatures for $\dot{M}_{*} = 10^{-8} M_{\odot}/yr$, "+r"$\alpha$="+str(params.alpha), fontsize = 15)
# axs[2].set_title(" Surface densities for $\dot{M}_{*} = 10^{-8} M_{\odot}/yr$, "+r"$\alpha$="+str(params.alpha), fontsize = 15)


plt.tight_layout()
plt.savefig('figures/H_R_t0_alpha'+str(params.alpha)+".png")

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (6,6))
params = code_gas.Params(**params_dict, H_r_model = "Liu_mixed")
params2 = code_gas.Params(**params_dict, H_r_model = "Lambrechts_mixed", epsilon_el=1, epsilon_heat=1)

print(params)
axs.loglog(r, sigma_gas_steady_state(r, t, params), color = 'black', linestyle = '-', label = "steady state")
axs.loglog(r, sigma_from_flux_general(r, t, flux_dtg_t(t, params), st_frag_drift(r, t, params), params), color = 'red', linestyle = '-')
axs.axvline(iceline(t, 1200, params))
axs.axvline(iceline(t, 1200, params2), color = 'gold')
print(params2.epsilon_heat)
print(params.H_r_model)
axs.set_ylim(1e-3, 1e7)
axs.yaxis.set_major_locator(mticker.LogLocator(numticks=999))
axs.yaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs="auto"))